In [ ]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
persones=pd.read_csv('Accident_people_2021_2022.csv')
persones.info()

In [ ]:
weather=pd.read_csv('data_meteo_estudifinal_2021_2022.csv')
weather = weather.drop(weather.columns[0], axis=1)
weather.rename(columns={'DATA_LECTURA':'Date'},inplace=True)
weather.info()

In [ ]:
persones= persones[persones['Person_type'] == 'Driver']
persones.head()

In [ ]:
#FROM ALL THE DATA WE ARE JUST GOING TO FOCUS N¡ON THE DRIVERS BECAUSE
#THE PASSANGER OR THE PEDESTRIAN DOESN0T HAVE ANY KIND OF POWER TO NOT BE IN THE SITUATION OR HAVE ANY CONTROL

persones=persones[['Day_shift','Vehicle_type','Gender','Age','Date']]
weather=weather[['Date','03.TM','06.HRM','10.VVM10']]

#JOIN THE DATASETS WITH THE VARIABLE DATE
variables = pd.merge(persones, weather, on='Date', how='left')
variables.info()


In [ ]:
#add variable quarter and start studying the different variables
variables['Quarter'] = variables['Date'].apply(lambda x: pd.Timestamp(x).quarter)
variables.head()

In [ ]:
# LETS TRANSFORM TO CATHEGORIC THE COLUMN EDAT
#/// desconegut(404) -->NOT KNOWN
#transform value desconegut a 404
variables['Age'] = variables['Age'].replace('Desconegut', '404')
def categorize_age(age):
    if int(age) <= 25:
        return '16-25'
    elif int(age) <= 35 and int(age) > 25:
        return '26-35'
    elif int(age) <= 45 and int(age) > 35:
        return '36-45'
    elif int(age) <= 55 and int(age) > 45:
        return '46-55'
    elif age == '404':
        return 'Age not known'
    else:
        return '+56'

# Apply function to transform ages to categories
variables['Age'] = variables['Age'].apply(categorize_age)

x=variables['Age'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['Age'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

In [ ]:
#x=variables['03.TM'].unique()
# print(np.sort(x)) the data goes from 3.8 to 34.2
def cat_temp(temp):
    if int(temp) <= 13:
        return 'Very low'
    elif int(temp) <= 17 and int(temp) > 13:
        return 'Low'
    elif int(temp) <= 24 and int(temp) > 17:
        return 'Mild'
    else:
        return 'Hot'
variables['03.TM'] = variables['03.TM'].apply(cat_temp)

x=variables['03.TM'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['03.TM'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

In [ ]:
#x=variables['06.HRM'].unique()
#print(np.sort(x)) from 9 percent to 100 percent
def cat_humidity(hum):
    if int(hum) <= 48:
        return 'Low humidity'
    elif int(hum) <= 60 and int(hum) > 48:
        return 'Moderate humidity'
    elif int(hum) <= 75 and int(hum) > 60:
        return 'High humidity'
    else:
        return 'Very high'
variables['06.HRM'] = variables['06.HRM'].apply(cat_humidity)

x=variables['06.HRM'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['06.HRM'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

In [ ]:
#x=variables['10.VVM10'].unique()
#print(np.sort(x)) from 0 to 8.7
def cat_wind(wind):
    if int(wind) <= 1:
        return 'Low speed'
    elif int(wind) <= 2 and int(wind) > 1:
        return 'Moderate speed'
    else:
        return 'High speed'
    
variables['10.VVM10'] = variables['10.VVM10'].apply(cat_wind)

x=variables['10.VVM10'].unique()
print(x)

# count the frequency of each value in column 'a'
value_counts = variables['10.VVM10'].value_counts()

# calculate the percentage of times each value appears
percentage = (value_counts / len(variables)) * 100
print(percentage)

In [ ]:
from itertools import combinations
from sklearn import preprocessing
from kmodes.kmodes import KModes
# List of variable names
variables_name = ['Day_shift', 'Vehicle_type', 'Gender', 'Age', '03.TM', '06.HRM', '10.VVM10', 'Quarter']

# Generate combinations of 4 to 8 variables
combinations_list = []
for r in range(4, 9):
    combinations_list.extend(combinations(variables_name, r))

# Empty DataFrame to store results
table = pd.DataFrame()

def tuple_to_list(t):
    new_list = []
    for element in t:
        new_list.append(element)
    return new_list

# Iterate over combinations
for selected_columns in combinations_list:
    # Select columns from 'variables' DataFrame based on the combination
    listavar = tuple_to_list(selected_columns)
    subgrup_1 = variables[listavar]
    print(subgrup_1.shape[1])
    # Clustering code
    subgroup = subgrup_1.copy()
    from sklearn import preprocessing  
    le = preprocessing.LabelEncoder()
    subgroup_encoded = subgroup.apply(le.fit_transform)
    #to create the list to get the percenge of significant clusters
    l=[]
    # Iterate through cluster numbers
    for n_clusters in range(3, 19):
        # Clustering
        km = KModes(n_clusters=n_clusters, init='Cao', n_init=1, verbose=1)
        clusters = km.fit_predict(subgroup_encoded)

        # Combine with original data
        combined = pd.concat([subgroup.reset_index(), pd.DataFrame({'Cluster': clusters})], axis=1).drop('index', axis=1)

        # Create table
        table = pd.DataFrame(columns=range(n_clusters))

        # Fill in table with percentage of instances of each value in each cluster for each variable
        for var in subgroup.columns:
            by_cluster_value = combined.groupby(['Cluster', var]).size()
            percentages = by_cluster_value.groupby(level=0).apply(lambda x: x / x.sum() * 100)
            pivot_table = pd.pivot_table(percentages.reset_index(), index=var, columns='Cluster', values=0, fill_value=0)

            # Fill in table with percentage of instances for each value in each cluster
            for i, col in enumerate(pivot_table.columns):
                values = pivot_table[col].index.tolist()
                pct_values = pivot_table[col].tolist()
                for j, value in enumerate(values):
                    table.loc[value, i] = pct_values[j]
            # Add new column with variable names based on index
        table['VARIABLE'] = ''

        variable_mapping = {}

        for column in subgroup.columns:
            unique_values = subgroup[column].unique()
            variable_mapping[column] = unique_values

        for i, index_value in enumerate(table.index.values):
            for variable, values in variable_mapping.items():
                if index_value in values:
                    table.iloc[i, -1] = variable
                    break
        print(table)
        table = table.reset_index().rename(columns={'index': 'value'})
        table = table[['VARIABLE'] + list(table.columns[:-1])]
        #print(table)
        table = table.set_index('VARIABLE')       

        # Add a new row for the cluster drivers percentage
        table.loc['Cluster percentage of drivers'] = 0

        # Calculate the number of instances in each cluster
        cluster_counts = combined['Cluster'].value_counts()

        # Fill in the table with the percentage of instances in each cluster
        for i, count in cluster_counts.items():
            table.loc['Cluster percentage of drivers', i] = count / len(combined) * 100
    
        #table.to_excel(f'table_{n_clusters}_clusters_pas1.xlsx', index=True)
        table1=table
        # Save table to CSV file
        table = table.drop('value', axis=1)
        new_table_2 = pd.DataFrame(index=table.index.unique())
    # Iterate over each column (0, 1, 2, 3) and find the highest percentage for each variable
        for col in table.columns:
            col_max = table.groupby(level=0)[col].max()
            new_table_2[col] = col_max
    # Add the 'Cluster percentage of drivers' row back to the new DataFrame
        new_table_2.loc['Cluster percentage of drivers'] = table.loc['Cluster percentage of drivers']
    # Display the new DataFrame
        #print(new_table_2)
        #new_table_2.to_excel(f'table_{n_clusters}_clusters_pas2.xlsx', index=True)
        new_table_3= new_table_2.loc[new_table_2.index != 'Cluster percentage of drivers'] = (new_table_2.loc[new_table_2.index != 'Cluster percentage of drivers'] >= 75).astype(int)
        new_table_3.loc['VARIABLES => 75%'] = new_table_3.sum()
        #new_table_2.to_excel(f'table_{n_clusters}_clusters_pas3.xlsx', index=True)
        #print(new_table_3)
        print(subgrup_1.shape[1])  
        new_table_3.loc['Cluster Value'] = new_table_3 .loc['VARIABLES => 75%'] / subgrup_1.shape[1]
        new_table_3.loc['Cluster adequate?'] = new_table_3 .loc['Cluster Value']>0.5
        new_table_4=new_table_3
        new_table_4.loc['Cluster percentage of drivers'] = table.loc['Cluster percentage of drivers']
        new_table_4.loc['SCORE'] = np.nan
        new_table_4.loc['SCORE' ,'TOTAL CLUSTERIZATION VALUE'] = new_table_4.loc['Cluster adequate?'].sum() / n_clusters
        if new_table_4.loc['Cluster adequate?'].sum() / n_clusters > 0.9:
            table1.to_excel(f'table_{n_clusters}_clusters_{"".join(selected_columns)}.xlsx', index=True)
       
    
    